In [1]:
#Primer paso que se realizo fue importar todas las librerias que vamos a utilizar
import pandas as pd
import json
import re
from fastapi import FastAPI
from datetime import datetime
import ast
from fastapi import FastAPI
import uvicorn

In [3]:
df=pd.read_csv(r"C:\Users\Equipo\Downloads\Csv_Proyecto_Terminado.csv")

In [4]:
df.head(2)

,budget,genres,id,overview,popularity,production_companies,production_countries,release_year,revenue,runtime,...,title,vote_average,vote_count,return,actors,director,month,dia_numero,nombre_dia_espanol,anio
0,30000000.0,Animation,862.0,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,...,Toy Story,7.7,5415.0,12.451801,Tom Hanks,John Lasseter,octubre,30,Lunes,1995.0
1,65000000.0,Adventure,8844.0,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,262797249.0,104.0,...,Jumanji,6.9,2413.0,4.043035,Robin Williams,Larry J. Franco,diciembre,15,Viernes,1995.0


In [6]:
# Crear una instancia de FastAPI
app = FastAPI()

In [20]:
@app.get('/peliculas_duracion')
def peliculas_duracion(pelicula: str):
    pelicula_encontrada = df[df["title"] == pelicula]
    if not pelicula_encontrada.empty:
        duracion = pelicula_encontrada["runtime"].values[0]
        año = pelicula_encontrada["anio"].values[0]
        return f"{pelicula}. Duración: {duracion}. Año: {año}"
    else:
        return "Película no encontrada"

In [51]:
import ast

@app.get('/franquicia')
def franquicia(franquicia: str):
    df['production_companies'] = df['production_companies'].apply(ast.literal_eval)
    peliculas_franquicia = df.loc[df['production_companies'].apply(lambda x: franquicia in x)]
    cantidad_peliculas = len(peliculas_franquicia)
    ganancia_total = peliculas_franquicia['revenue'].sum()
    ganancia_promedio = ganancia_total / cantidad_peliculas if cantidad_peliculas > 0 else 0
    return f"La franquicia {franquicia} posee {cantidad_peliculas} películas, una ganancia total de {ganancia_total} y una ganancia promedio de {ganancia_promedio}"


In [41]:
@app.get('/peliculas_idioma')
def peliculas_idioma(idioma: str):
    cantidad_peliculas = sum(1 for _, pelicula in df.iterrows() if isinstance(pelicula["spoken_languages"], str) and idioma in pelicula["spoken_languages"])
    return f"{cantidad_peliculas} cantidad de películas fueron estrenadas en {idioma}"




In [30]:
@app.get('/peliculas_pais')
def peliculas_pais(pais: str):
    cantidad_peliculas = sum(1 for index, pelicula in df.iterrows() if isinstance(pelicula["production_countries"], str) and pais in pelicula["production_countries"].split(","))
    return f"Se produjeron {cantidad_peliculas} películas en el país {pais}"




In [24]:
@app.get('/productoras_exitosas')
def productoras_exitosas(productora: str):
    peliculas_productora = df[df["production_companies"].str.contains(productora, na=False)]
    cantidad_peliculas = len(peliculas_productora)
    revenue_total = peliculas_productora["revenue"].sum()
    return f"La productora {productora} ha tenido un revenue de {revenue_total} y ha realizado {cantidad_peliculas} películas"


In [7]:
@app.get('/get_director/{nombre_director}')
def get_director(nombre_director: str):
    director_movies = df[df['director'] == nombre_director]

    # Calcular la métrica de éxito del director
    exito_director = calcular_exito_director(director_movies)

    # Recopilar información detallada de cada película
    movies_info = []
    for _, movie in director_movies.iterrows():
        movie_info = {
            'nombre_pelicula': movie['title'],
            'fecha_lanzamiento': movie['release_year'],
            'retorno_individual': movie['return'],
            'costo': movie['budget'],
            'ganancia': movie['revenue']
        }
        movies_info.append(movie_info)

    return {
        'exito_director': exito_director,
        'peliculas': movies_info
    }

# Función para calcular la métrica de éxito del director
def calcular_exito_director(director_movies):
    # Implementa tu lógica para calcular la métrica de éxito del director
    # Por ejemplo, puedes calcular el promedio de puntuaciones de las películas del director
    exito_director = director_movies['vote_average'].mean()
    return exito_director

In [ ]:

@app.get('/recomendacion')
async def recomendacion(titulo: str):
    # Convertir el título proporcionado a minúsculas
    titulo = titulo.lower()

    # Obtener la fila correspondiente al título proporcionado
    pelicula = df.loc[df['title'].str.lower() == titulo]

    if pelicula.empty:
        return []

    # Obtener la puntuación de la película
    puntuacion = pelicula['vote_average'].values[0]

    # Encontrar películas similares según la puntuación
    peliculas_similares = df.loc[df['vote_average'] >= puntuacion].sort_values('vote_average', ascending=False)

    # Obtener los títulos de las 5 películas con puntuacion mas similar
    recomendaciones = peliculas_similares['title'].head(5).tolist()

    return recomendaciones

if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000)


In [54]:
#Ultimo push para guardar el dataset listo
df.to_csv('Csv_Proyecto_Terminado.csv', index=False)